# Zadanie 6

Celem ćwiczenia jest implementacja algorytmu Q-learning.

Następnie należy stworzyć agenta rozwiązującego problem [Taxi](https://gymnasium.farama.org/environments/toy_text/taxi/). Problem dostępny jest w pakiecie **gym**.

Punktacja (max 8 pkt):
- Implementacja algorytmu Q-learning. [3 pkt]
- Eksperymenty dla różnych wartości hiperparametrów [2 pkt]
- Jakość kodu [1.5 pkt]
- Wnioski [1.5 pkt]


In [1]:
import numpy as np
import random
import gym
import pygame
from IPython.display import clear_output
from time import sleep

random.seed(1234)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Interfejs

class QLearningSolver:
    """Class containing the Q-learning algorithm that might be used for different discrete environments."""
    def __init__(self,
                 streets:any,
                 learning_rate:float=0.1,
                 gamma:float=0.9, #discount rate
                 epsilon:float=0.1, #exploration rate
                 ):
        self.streets = streets
        self.observation_space = streets.observation_space.n
        self.action_space = streets.action_space.n

        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon

        self.q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

    def __call__(self, state:np.ndarray, action:np.ndarray) -> np.ndarray:
        """Return Q-value of given state and action."""
        return self.q_table[state, action]

    def update(self, state:np.ndarray, action:np.ndarray, next_state:np.ndarray, reward:float) -> None: #reward <0 to kara tak jakby
        """Update Q-value of given state and action."""

        val = reward + self.gamma * np.max(self.q_table[next_state])

        prev_q = (1-self.learning_rate) * self(state, action) # multiplied by previous reward 

        new_val = prev_q + self.learning_rate * val #dla 0.1 alg. w stopniu nieznacznym uwzględnia kolejny ruch
        self.q_table[state, action] =  new_val

    def get_best_action(self, state:np.ndarray) -> np.ndarray:
        """Return action that maximizes Q-value for a given state."""
        return np.argmax(self.q_table[state])

    def __repr__(self):
        """Elegant representation of Q-learning solver."""

    def learn(self, episodes):
        temp = []
        for episode in range(episodes):
            state = self.streets.reset()
            done = False
            trip_length = 0
    
            while not done and trip_length < 25:
                random_value = random.uniform(0, 1)
                if (random_value < self.epsilon):
                    action = self.streets.action_space.sample() # Explore a random action
                else:
                    action = self.get_best_action(state) # Use the action with the highest q-value
            
                next_state, reward, done, info = self.streets.step(action)

                self.update(state, action, next_state, reward)
            
                trip_length += 1
                state = next_state
            temp.append(trip_length)
        for i in range(10):
            print(np.mean(temp[1000 * i:1000 * (i+1)]))

    def solve(self, initial_state):
        self.q_table[initial_state]


        lengths=[]
        for tripnum in range(1, 11):
            state = self.streets.reset()
            self.streets.render()

            done = False
            trip_length = 0
        
            while not done and trip_length < 25:
                action = self.get_best_action(state)
                next_state, reward, done, info = self.streets.step(action)
                clear_output(wait=True)
                print("Trip number " + str(tripnum) + " Step " + str(trip_length))
                print(self.streets.render(mode='ansi'))
                sleep(.2)
                state = next_state
                trip_length += 1
            lengths.append(trip_length)

            sleep(.2)
        avg_len=sum(lengths)/10
        return avg_len
        

    def __str__(self):
        return self.__repr__()

# Uczenie modelu na 10 000 iteracjach

In [3]:
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.1 #exploration rate


#streets.render()
solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

24.997
24.926
24.596
23.443
21.518
18.584
17.252
16.31
15.749
15.375


In [4]:
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.5
gamma:float=0.9 #discount rate
epsilon:float=0.1 #exploration rate


initial_state = streets.encode(1, 0, 2, 0)
streets.s = initial_state
solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

print(solver.solve(initial_state))


Trip number 10 Step 12
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

12.3


# Uczenie modelu na 30 000 iteracjach

In [5]:
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.5
gamma:float=0.9 #discount rate
epsilon:float=0.1 #exploration rate


initial_state = streets.encode(1, 0, 2, 0)
streets.s = initial_state
solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(30000)

print(solver.solve(initial_state))

Trip number 10 Step 10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

13.4


# Wnioski

# Learning rate
Parametr jest wykorzystywany do wartościowania jak szybko chcemy żeby algorytm się uczył. Jak bardzo wartościujemy nową wartość reward w porównaniu do poprzedniej
* zbyt duże zwiększenie lub zmieniejszenie tego parametru prowadzi do wstrzymania procesu uczenia i podejmowania decyzji które wahają się wokół optymalnych, ale nie ostatecznie nie dają najbardziej optymalnej drogi.

In [6]:
print("Learning rate = 0.0")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.0
gamma:float=0.6 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Learning rate = 0.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0


In [7]:
print("Learning rate = 0.1")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Learning rate = 0.1
24.995
24.929
24.476
23.158
20.792
17.271
15.752
14.842
14.047
13.992


In [8]:
print("Learning rate = 0.5") #optymalna
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.4
gamma:float=0.6 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Learning rate = 0.5
24.814
18.264
13.548
13.107
13.045
13.056
13.033
13.146
13.154
12.979


# Gamma
Parametr opowiada za odpowiednie wartościowanie(uwzględniając gamma jako wagę) najkorzystniejszego następnego ruchu, który może teraz wykonać alogrytm. Parametr odwierciedla horyzont wyborów, które podejmuje algorytm.
* parametr jeśli będzie zbyt mały nie będzie wogóle uwzględniał lub uwzględniał pomijalnie reward za kolejny ruch, przez co algorytm będzie skupiony jedynie na najbliższym ruchu, który w globalnym spojrzeniu nie zawsze jest najlepszym wyborem(jest tak jakby optimum lokalnym).
* jeśli będzie za duży będzie zbyt zoreintowany na przyszłych działaniach, dlaczego nie jest to zbyt dobre, ponieważ obecne wybory nie zawsze są znaczące dla znalezienia optymalnego rozwiązania.

In [9]:
print("Gamma = 0.0")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=0.0 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Gamma = 0.0
24.999
25.0
24.974
24.966
24.973
24.992
24.972
24.988
24.961
24.979


In [10]:
print("Gamma = 0.6")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Gamma = 0.6
25.0
24.924
24.609
23.685
20.969
17.88
15.577
14.801
14.062
13.629


In [11]:
print("Gamma = 0.9") #optymalna
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=1.0 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

Gamma = 0.9
25.0
24.892
23.507
20.126
14.689
13.266
13.042
13.157
13.039
13.044


# Epsilon
Parametr opowiada za wybór wartości(akcji), która jest ruchem taksówki na planszy.
* jeśli mamy mały parametr to, dużej większości powinien być wybierany ruch najkorzystniejszy dla garacza.
* parametr jest wykorzystywany do zwiększenie szybkości poszukiwania najkorzystniejszej drogi, ponieważ wybierany tylko
czasem parametr może zwiększyć możliwości eksploracyjne algorytmu, przez co przyspieszyć zanjdywanie najkorzystniejszej drogi.
* jednakże zbyt duża wartość parametru może przyczynić się do zbyt dużej randomizacji wyboru kolejnego ruchu przez co droga, którą wybiera taksówka przestaje być optymalna i znacznie się wydłuża.

In [12]:
print("epsilon = 0.0")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4

# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.0 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

epsilon = 0.0
24.999
24.942
24.493
23.034
20.467
17.887
15.721
15.047
14.056
13.806


In [13]:
print("epsilon = 0.1")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4
# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.1 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

epsilon = 0.1
25.0
24.93
24.603
23.626
21.283
18.837
17.132
16.188
15.774
15.504


In [14]:
print("epsilon = 0.5")
streets = gym.make("Taxi-v3").env #New versions keep getting released; if -v3 doesn't work, try -v2 or -v4
# Hyperparameters
learning_rate:float=0.1
gamma:float=0.6 #discount rate
epsilon:float=0.5 #exploration rate

solver = QLearningSolver(streets, learning_rate, gamma, epsilon)

solver.learn(10000)

epsilon = 0.5
25.0
24.975
24.911
24.708
24.233
23.666
23.265
22.855
22.874
22.88
